<a href="https://colab.research.google.com/github/mugurm/colab/blob/main/mugurm_stock_explorer_colab_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Explorer (Automatic Trailing Stop edition)
by mugurm

In [49]:
# Import the necessary libraries
import yfinance as yf
import holoviews as hv
import pandas as pd
import numpy as np
from holoviews import opts
from ipywidgets import widgets, Layout
from datetime import datetime, timedelta
from IPython.display import display, clear_output

# Use bokeh backend
hv.extension('bokeh')    

def download_data(symbol, start, end):
    """Fetch stock data using yfinance library."""
    try:
        df = yf.download(symbol, start=start, end=end)
        return df
    except Exception as e:
        print(f"Failed to fetch data for {symbol}. Error: {str(e)}")
        return pd.DataFrame()  # Return an empty DataFrame on failure


def calculate_trailing_stops(df, trailing_stop_sell, trailing_stop_buy, offset, start_long):
    """Calculate the trailing stop levels based on the highest or lowest price reached so far."""
    if start_long:
        df['Trailing_Sell_Level'] = df['High'].expanding(min_periods=offset).max() * (1 - trailing_stop_sell / 100)
    else:  # start_short
        df['Trailing_Buy_Level'] = df['Low'].expanding(min_periods=offset).min() * (1 + trailing_stop_buy / 100)
    
    # ensure no trailing stop level before offset
    if start_long:
        df.loc[df.index[:offset], 'Trailing_Sell_Level'] = np.nan
    else:
        df.loc[df.index[:offset], 'Trailing_Buy_Level'] = np.nan
    
    return df


def find_buy_and_sell_signals(df, trailing_buy_pct, trailing_sell_pct, offset, start_long):
    """Find the buy and sell signals."""
    df_offset = df.iloc[offset:]  # only consider data from the offset day
    
    if start_long:
        buy_signal = (df_offset.Low <= df_offset.Trailing_Sell_Level)
    else:
        buy_signal = (df_offset.High >= df_offset.Trailing_Buy_Level)

    buy_date, buy_price, sell_date, sell_price = None, None, None, None

    if buy_signal.any():
        buy_date = buy_signal.idxmax()  # date of first True
        buy_price = df.loc[buy_date, 'Low'] if start_long else df.loc[buy_date, 'High']
        df_after_buy = df.loc[buy_date:].copy()

        # Get the next date after buy_date, and terminate the "Trailing_Sell_Level" curve from that date onwards
        if len(df.loc[buy_date:].index) > 1:
            next_day = df.loc[buy_date:].index[1]
            df.loc[next_day:, 'Trailing_Sell_Level' if start_long else 'Trailing_Buy_Level'] = np.nan

        # Terminate the "Trailing_Sell_Level" curve in the original df and df_after_buy from buy_date onwards
        df_after_buy['Trailing_Sell_Level' if start_long else 'Trailing_Buy_Level'] = np.nan

        if start_long:
            df_after_buy['Trailing_Buy_Level'] = df_after_buy['Low'].expanding().min() * (1 + trailing_buy_pct / 100)
            sell_signal = (df_after_buy.High >= df_after_buy.Trailing_Buy_Level)
        else:
            df_after_buy['Trailing_Sell_Level'] = df_after_buy['High'].expanding().max() * (1 - trailing_sell_pct / 100)
            sell_signal = (df_after_buy.Low <= df_after_buy.Trailing_Sell_Level)
        
        if sell_signal.any():
            sell_date = sell_signal.idxmax()  # date of first True
            sell_price = df_after_buy.loc[sell_date, 'High'] if start_long else df_after_buy.loc[sell_date, 'Low']

            # Get the next date after sell_date, and terminate the "Trailing_Buy_Level" curve from that date onwards
            if len(df_after_buy.loc[sell_date:].index) > 1:
                next_day = df_after_buy.loc[sell_date:].index[1]
                df_after_buy.loc[next_day:, 'Trailing_Buy_Level' if start_long else 'Trailing_Sell_Level'] = np.nan

        df = df_after_buy.combine_first(df)  # give priority to df_after_buy and merge the new trailing levels into the original dataframe
    return buy_date, buy_price, sell_date, sell_price, df



def create_plot(df, buy_date, buy_price, sell_date, sell_price, symbol, buy_pct, sell_pct, offset, start_long):
    """Create and display the plot."""
    price_curve = hv.Curve(df, ('Date', 'Date'), ('High', 'High')).opts(color='lightgray') * \
                  hv.Curve(df, ('Date', 'Date'), ('Low', 'Low')).opts(color='lightgray')
    sell_curve = hv.Curve(df, ('Date', 'Date'), ('Trailing_Sell_Level', 'Trailing Sell Level')).opts(color='red')

    # Only plot Trailing_Buy_Level if it exists
    if 'Trailing_Buy_Level' in df.columns:
        buy_curve = hv.Curve(df, ('Date', 'Date'), ('Trailing_Buy_Level', 'Trailing Buy Level')).opts(color='green')
        # Create and add the buy dotted line
        buy_line = hv.Curve([(buy_date, buy_price), (buy_date, df.loc[buy_date, ('Trailing_Buy_Level' if start_long else 'Trailing_Sell_Level')])]).opts(color='green' if start_long else 'red', line_dash='dotted')
    else:
        buy_curve = hv.Curve([])  # Empty placeholder to not disrupt the plot
        buy_line = hv.Curve([])  # Empty placeholder to not disrupt the plot

    # Create and add the sell dotted line
    offset_date = df.index[offset]
    sell_line = hv.Curve([(offset_date, df.loc[offset_date, 'High']), (offset_date, df.loc[offset_date, ('Trailing_Sell_Level' if start_long else 'Trailing_Buy_Level')])]).opts(color='red' if start_long else 'green', line_dash='dotted')

    buy_point = hv.Scatter([(buy_date, buy_price)] if buy_date else []).opts(color='red' if start_long else 'green', size=10)
    sell_point = hv.Scatter([(sell_date, sell_price)] if sell_date else []).opts(color='green' if start_long else 'red', size=10)

    # Add yellow dot for the first day of the first trailing stop calculation
    first_trailing_stop_point = hv.Scatter([(offset_date, df.loc[offset_date, 'High'])]).opts(color='orange', marker='square', size=10)

    # Construct the title string
    title_str = f"{symbol} - Buy: {buy_pct}%, Sell: {sell_pct}%"

    display((price_curve * sell_curve * buy_curve * buy_line * sell_line * buy_point * sell_point * first_trailing_stop_point).opts(width=800, height=350, title=title_str))


def plot_stock(symbol, days, trailing_stop_pct_buy, trailing_stop_pct_sell, days_offset, start_long):
    """Fetch and plot the stock data."""
    end = datetime.now()
    start = end - timedelta(days=days)

    # Fetch data
    df = download_data(symbol, start, end)
    if df.empty:
        print(f"No data available for {symbol} in the given date range.")
        return

    # Calculate trailing stops
    df = calculate_trailing_stops(df, trailing_stop_pct_sell, trailing_stop_pct_buy, days_offset, start_long)

    # Find buy and sell signals
    buy_date, buy_price, sell_date, sell_price, df = find_buy_and_sell_signals(df, trailing_stop_pct_buy, trailing_stop_pct_sell, days_offset, start_long)
    
    # Create and display the plot
    create_plot(df, buy_date, buy_price, sell_date, sell_price, symbol, trailing_stop_pct_buy, trailing_stop_pct_sell, days_offset, start_long)


# Create input widgets
iwidth = '700px'
symbol_input = widgets.Text(value='GOOG', description='Stock Symbol:')
days_input = widgets.IntSlider(layout=Layout(width=iwidth), value=180, min=20, max=360, step=10, description='Days:')
days_offset = widgets.IntSlider(layout=Layout(width=iwidth), value=10, min=0, max=119, step=1, description='Start day:')
trailing_stop_pct_buy_input = widgets.FloatSlider(layout=Layout(width=iwidth), value=10.0, min=0.1, max=25.0, step=0.1, description='Buy Trailing Stop (%):')
trailing_stop_pct_sell_input = widgets.FloatSlider(layout=Layout(width=iwidth), value=15.0, min=0.1, max=25.0, step=0.1, description='Sell Trailing Stop (%):')
start_state_input = widgets.Checkbox(value=True, description='Start Long (off for Short):')


# Function to update the maximum value of days_offset slider dynamically
def update_days_offset_range(*args):
    # Using magic number to approximate trading days to calendar days.
    days_offset.max = int(round(days_input.value * 0.67))
    days_offset.value = 3

# Observe the value of the Days slider
days_input.observe(update_days_offset_range, 'value')

# Define the event handler
def on_input_change(change):
    clear_output(wait=True)
    display(symbol_input, days_input, trailing_stop_pct_buy_input, trailing_stop_pct_sell_input, days_offset, start_state_input)
    plot_stock(symbol_input.value, days_input.value, trailing_stop_pct_buy_input.value, trailing_stop_pct_sell_input.value, days_offset.value, start_state_input.value)

# Attach the event handler to the 'on_submit' event and 'value' change event
symbol_input.on_submit(on_input_change)
days_input.observe(on_input_change, names='value')
days_offset.observe(on_input_change, names='value')
trailing_stop_pct_buy_input.observe(on_input_change, names='value')
trailing_stop_pct_sell_input.observe(on_input_change, names='value')
start_state_input.observe(on_input_change, names='value')

# Display the widgets and initial plot
display(symbol_input, days_input, trailing_stop_pct_buy_input, trailing_stop_pct_sell_input, days_offset, start_state_input)
plot_stock(symbol_input.value, days_input.value, trailing_stop_pct_buy_input.value, trailing_stop_pct_sell_input.value, days_offset.value, start_state_input.value)



Text(value='GOOG', description='Stock Symbol:')

IntSlider(value=180, description='Days:', layout=Layout(width='700px'), max=360, min=20, step=10)

FloatSlider(value=17.7, description='Buy Trailing Stop (%):', layout=Layout(width='700px'), max=25.0, min=0.1)

FloatSlider(value=6.3, description='Sell Trailing Stop (%):', layout=Layout(width='700px'), max=25.0, min=0.1)

IntSlider(value=7, description='Start day:', layout=Layout(width='700px'), max=119)

Checkbox(value=False, description='Start Long (off for Short):')

[*********************100%***********************]  1 of 1 completed


:Overlay
   .Curve.I     :Curve   [Date]   (High)
   .Curve.II    :Curve   [Date]   (Low)
   .Curve.III   :Curve   [Date]   (Trailing_Sell_Level)
   .Curve.IV    :Curve   [Date]   (Trailing_Buy_Level)
   .Curve.V     :Curve   [x]   (y)
   .Curve.VI    :Curve   [x]   (y)
   .Scatter.I   :Scatter   [x]   (y)
   .Scatter.II  :Scatter   [x]   (y)
   .Scatter.III :Scatter   [x]   (y)

**YAHOO FINANCE FORMAT STOCK SYMBOLS: MSFT, GOOG, BTC-USD, ARSUSD=X**

You are a highly skilled, staff Google engineer and expert in python and data visualization. You are also very familiar with stock market trading and technical analysis and have programmed many automated trading systems and simulations. You know all of the concepts on Investopedia. You will help me add a new yellow dot using hv.Scatter to indicate the price on the first day of the first trailing stop calculation to the following Google Colab notebook: